In [284]:
# Import packages
import pandas as pd
import numpy as np

In [285]:
# Import data
tariffs = pd.read_csv('tariffs.csv')
trade = pd.read_csv('trade.csv')
pwt = pd.read_excel('pwt110.xlsx')
income_groups = pd.read_csv('income_groups.csv')

The following datasets are used:  
Tariffs: Tariff data (Source: World Bank's Integrated Trade Solution, UNCTAD TRAINS).  
Trade: Exports & Imports of goods and services as percent of GDP (World Bank).  
PWT: the Penn World Table, contains various economic data.  
Income Groups: World Bank's income classification, categorised based on countries' gross national income (GNI) per capita. 

For the analysis, I will be looking at weighted average tariffs. These are import-value-weighted average tariffs (%) per year. 

# Data Exploration

In [286]:
# Look at tariff data
print(tariffs.columns)

# Print unique years
years_trf = tariffs['Tariff Year']
years_trf = years_trf.sort_values(ascending=True)
years_trf = years_trf.unique()
print(years_trf)
print(f'Total years: {len(years_trf)}')

# Print unique countries
countries_trf = tariffs['Reporter Name'].unique()
print(f'Total countries: {len(countries_trf)}')

Index(['Reporter', 'Product', 'Partner', 'Tariff Year', 'DutyType',
       'Selected Nomen', 'Native Nomen', 'Reporter Name', 'Product Name',
       'Partner Name', 'Trade Year', 'Trade Source', 'Simple Average',
       'Simple Tariff Line Average', 'Weighted Average', 'Variance',
       'Standard Deviation', 'Sum Of Rates', 'Sum Of SAvgRates',
       'Count_Of_SAvgRates_Cases', 'Sum_Of_Squared_Rates', 'Minimum Rate',
       'Maximum Rate', 'Nbr of AVE Lines', 'Nbr of NA Lines',
       'Nbr of Free Lines', 'Nbr of Dutiable Lines', 'Nbr of Total Lines',
       'Nbr of DomesticPeaks', 'Nbr of InternationalPeaks', 'Nbr Line 0 to 5',
       'Nbr Line 5 to 10', 'Nbr Line 10 to 20', 'Nbr Line 20 to 50',
       'Nbr Line 50 to 100', 'Nbr Line more than 100', 'SumRateByWghtTrdValue',
       'SumWghtTrdValue4NotNull', 'Imports Value in 1000 USD',
       'Free Imports in 1000 USD', 'Dutiable Imports in 1000 USD',
       'Specific Duty Imports in 1000 USD', 'Binding Coverage'],
      dtype='objec

In [287]:
# Look at trade data
print(trade.columns)
trade.head()

# Print unique countries
countries_trd = trade['Country Name'].unique()
print(f'Total countries: {len(countries_trd)}')

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '1975 [YR1975]', '1976 [YR1976]', '1977 [YR1977]', '1978 [YR1978]',
       '1979 [YR1979]', '1980 [YR1980]', '1981 [YR1981]', '1982 [YR1982]',
       '1983 [YR1983]', '1984 [YR1984]', '1985 [YR1985]', '1986 [YR1986]',
       '1987 [YR1987]', '1988 [YR1988]', '1989 [YR1989]', '1990 [YR1990]',
       '1991 [YR1991]', '1992 [YR1992]', '1993 [YR1993]', '1994 [YR1994]',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]',
       '2019 [YR2019]', '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]',
       '2023 [YR2023]', '2024 [

In [288]:
# Look at economic data
print(pwt.columns)
print(pwt['year'].unique())

# Print unique countries
countries_pwt = pwt['country'].unique()
print(f'Total countries: {len(countries_pwt)}')

Index(['countrycode', 'country', 'currency_unit', 'year', 'rgdpe', 'rgdpo',
       'pop', 'emp', 'avh', 'hc', 'ccon', 'cda', 'cgdpe', 'cgdpo', 'cn', 'ck',
       'ctfp', 'cwtfp', 'rgdpna', 'rconna', 'rdana', 'rnna', 'rkna', 'rtfpna',
       'rwtfpna', 'labsh', 'irr', 'delta', 'xr', 'pl_con', 'pl_da', 'pl_gdpo',
       'i_cig', 'i_xm', 'i_xr', 'i_outlier', 'i_irr', 'cor_exp', 'csh_c',
       'csh_i', 'csh_g', 'csh_x', 'csh_m', 'csh_r', 'pl_c', 'pl_i', 'pl_g',
       'pl_x', 'pl_m', 'pl_n', 'pl_k'],
      dtype='object')
[1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963
 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977
 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991
 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005
 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2021 2022 2023]
Total countries: 185


In [289]:
# Look at income groups data
countries_inc = income_groups['Entity'].unique()

In [290]:
# Print intersecting countries of datasets
countries_intersection = list(set(countries_trf) & set(countries_trd) & set(countries_pwt) & set(countries_inc))
countries_intersection.sort()
print(f'Countries in all 3 datasets: {len(countries_intersection)}')
print(countries_intersection)

Countries in all 3 datasets: 144
['Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica', 'Croatia', 'Cyprus', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Eswatini', 'Fiji', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Lithuania', 'Luxembou

# Data Cleaning

In [291]:
# Clean tariff data
tariffs = tariffs[['Reporter Name', 'Tariff Year', 
                   'Weighted Average', 'Standard Deviation']].rename(columns={
    'Reporter Name': 'country',
    'Tariff Year': 'year',
    'Weighted Average': 'tariff',
    'Standard Deviation': 'tariff_sd'
})

In [292]:
# Clean trade data
# Original dataset encodes years as '1980 [YR1980]' etc.
raw_year_cols = [col for col in trade.columns if '[' in col]

# Extract numeric year from each column name
clean_year_map = {col: col.split(' ')[0] for col in raw_year_cols}
trade = trade.rename(columns = clean_year_map)

# Identify years
year_columns = [col for col in trade.columns if col.isdigit()]

trade = trade.melt(
    id_vars = 'Country Name',
    value_vars = year_columns,
    var_name = 'Year',
    value_name = 'trade_gdp'
)

trade['Year'] = trade['Year'].astype(int)

In [293]:
# Clean income group data
income_groups = income_groups.rename(columns = {'Entity': 'country',
                                                'Year': 'year',
                                                'World Bank\'s income classification': 'income_group'})
income_groups = income_groups.drop(columns = ['Code'])
income_groups.head()

,country,year,income_group
0,Afghanistan,1987,Low-income countries
1,Afghanistan,1988,Low-income countries
2,Afghanistan,1989,Low-income countries
3,Afghanistan,1990,Low-income countries
4,Afghanistan,1991,Low-income countries


In [294]:
# Clean PWT data
pwt = pwt.filter(['country', 'year', 'rgdpe', 'rgdpo', 'pop', 'hc'])
pwt = pwt.sort_values(['country', 'year'])

# Compute Real GDP per capita
pwt['rgdp_pc'] = pwt['rgdpe'] / pwt['pop']

# Compute GDP per capita growth
pwt['log_gdppc_growth'] = (np.log(pwt['rgdp_pc']) 
                                - np.log(pwt.groupby('country')['rgdp_pc'].shift(1))
                                ) * 100


# Merge datasets

In [295]:
panel.head()

,country,year,log_gdppc_growth,tariff,tariff_sd,trade_gdp,income_group
0,Albania,1997,-20.280174,19.35,7.81,44.7293780649181,Low-income countries
1,Albania,1997,-20.280174,13.59,5.96,44.7293780649181,Low-income countries
2,Albania,1997,-20.280174,7.79,5.50,44.7293780649181,Low-income countries
3,Albania,1997,-20.280174,12.37,5.48,44.7293780649181,Low-income countries
4,Albania,2000,10.814410,10.36,5.82,61.6092610436407,Lower-middle-income countries


In [296]:
income_groups.head()

,country,year,income_group
0,Afghanistan,1987,Low-income countries
1,Afghanistan,1988,Low-income countries
2,Afghanistan,1989,Low-income countries
3,Afghanistan,1990,Low-income countries
4,Afghanistan,1991,Low-income countries


In [297]:
# Rename columns
trade = trade.rename(columns={'Country Name': 'country', 'Year': 'year'})

# Merge datasets 
panel = pwt.merge(trade, on=['country', 'year'], how='left')
panel = panel.merge(tariffs, on=['country', 'year'], how='left')
panel = panel.merge(income_groups, on=['country', 'year'], how='left')

# Drop rows with missing data
panel = panel.dropna(subset=['log_gdppc_growth', 'tariff', 'trade_gdp', 'income_group'])
panel = panel.reset_index(drop=True)

# Keep relevant variables
panel = panel[['country', 'year', 'log_gdppc_growth', 'tariff', 'tariff_sd', 'trade_gdp', 'income_group']]

In [298]:
# Export dataset
panel.to_csv('panel_data.csv')